# Fundações

Esse notebook conterá o dimensionamento a fundações de sapatas conforme NBR 6120. 

## Dimensionamento a flexão (Fiz isso aqui como exemplo só para vc entender como quero que fica, faz as suas coisas depois apaga isso)

### Introdução

O dimensionamento de vigas a flexão é um processo fundamental na engenharia estrutural, utilizado para garantir a segurança e estabilidade das construções. O conceito básico envolve calcular as dimensões de uma viga (altura e largura) de modo que ela suporte os momentos e forças atuantes sem falhas. Para o dimensionamento de uma viga retangular, a tensão de flexão máxima não pode ultrapassar a tensão admissível do material. O cálculo da largura $ b $ e da altura $ h $ é feito considerando a relação entre a carga, o momento e a capacidade de resistência do material.

Se $ f_{\text{cd}} $ é a resistência característica do concreto e $ f_{\text{yd}} $ é a resistência do aço, a condição de segurança é dada por:

$$
\sigma_{\text{max}} \leq f_{\text{cd}}
$$

Ou seja, a tensão de flexão máxima $ \sigma_{\text{max}} $ deve ser inferior à resistência admissível do concreto $ f_{\text{cd}} $.


### Função python que calcula

In [19]:
import numpy as np
import pandas as pd
import math

def dimensionar_viga(m_sd: float, f_cd: float, b_max: float = 0.3) -> float:
    """
    Calcula o dimensionamento da altura de uma viga retangular a partir do momento fletor máximo.

    :param m_sd: Momento fletor máximo (Nm).
    :param f_cd: Resistência característica do concreto (MPa).
    :param b_max: Largura máxima da viga (m). Default é 0.3 m.

    :return: A altura da viga (m) necessária para suportar o momento fletor.
    
    A fórmula usada para calcular a altura é:
    h = (6 * m_sd / (f_cd * b_max))**(1/3)
    """
    # Convertendo f_cd para N/m²
    f_cd *= 1e6
    
    # Cálculo da altura da viga
    h = (6 * m_sd / (f_cd * b_max))**(1/3)
    
    return h


### Exemplo

In [4]:
# Exemplo de uso da função
m_s = 5000  # Momento fletor em Nm
f_cd = 25  # Resistência característica do concreto em MPa

altura_viga = dimensionar_viga(m_s, f_cd)
print(f"A altura necessária da viga é: {altura_viga:.2f} metros")

A altura necessária da viga é: 0.16 metros


## Dimensionamento de Sapatas


Este notebook apresenta o dimensionamento de fundações superficiais do tipo sapata isolada conforme NBR 6120:2019 e NBR 6118:2023. A aplicação desenvolvida realiza o dimensionamento e e cria ferramentas para otimização com base na verificação de restrições normativas e geométricas, aplicando penalidade às verificaçõe que não atendem aos requisitos de projeto.


### Introdução

As sapatas são elementos de fundações superficiais utilizados para transmitir os esforços provenientes da superestrutura para o solo. Segundo BASTOS (2023), representam um dos elementos mais comuns para esta finalidade. O dimensionamento adequado depende tanto das características geotécnicas do solo, como tambem das características geométricas e de carregamento atuantes. 
Alem disso, como este trabalho visa manipular todos os elementos de fundação presentes em um projeto simultaneamente, a verificação de sobreposição dos elementos é essencial para a viabilidade do projeto. Este trablho aborda todas esses pontos de forma automatizada para alcançar a otimização.

### Verificação

As verificações são implementadas por meio de restrições formuldas como funções de penalidade. Cada verificação é representada por uma função g, cuja condição de atendimento é dada por:
$$
g \leq 0
$$
Onde:
* $g$: É a função de verificação (admensional).

Caso $g > 0$, compreende-se que a verificação não foi atendida e consequentemente, o elemento de fundação é penalizado.
<br>(Escrevi essa parte "verificação" pois é preciso ter, mas não esta finalizada)

## Tensão admissível do solo


Para o cálculo da tensão admissível do do solo, é necesário o uso de informações coletadas a partir do ensaio sondagem SPT (Standart Penetretion Test), especificadaemnte o N_spt e a classiificação do tipo de solo. 
Neste trabalho, adota-se uma formulação empírica baseada no método de estimativa simplificada, comumente empregada em estudos geotécnicos preliminares. A tensão admissível 𝜎adm,  é calculada conforme o tipo de solo, como segue:
* para argila:
$$
\sigma_{adm} = \frac{Nspt}{50} \cdot 1000
$$

* para areia:
$$
\sigma_{adm} = \frac{Nspt}{40} \cdot 1000
$$

* para pedregulho:
$$
\sigma_{adm} = \frac{Nspt}{30} \cdot 1000
$$

### Função python

In [17]:
def tensao_adm_solo(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calcula a tensão admissível do solo.

    Args:
        df (DataFrame): DataFrame com os dados de entrada, contendo as colunas 'spt' e 'solo'.

    Returns:
        DataFrame: DataFrame com a coluna 'sigma_adm (kPa)' calculada.
    """
    # Converta a coluna 'solo' para minúsculas
    solo_column = df[('solo')] 
    solo_column = solo_column.str.lower()
    
    
    # Verifique se a coluna 'spt' existe para evitar erro
    if 'spt' not in df.columns:
        raise KeyError("A coluna 'spt' deve estar presente no DataFrame.")

    # Calcula a tensão admissível com base no tipo de solo
    condicoes = [
        solo_column == 'pedregulho',
        solo_column == 'areia',
        (solo_column == 'silte') | (solo_column == 'argila'),
    ]
    values = [
        df[('spt')] / 30 * 1E3,  # Acessa a coluna 'spt' corretamente
        df[('spt')] / 40 * 1E3,
        df[('spt')] / 50 * 1E3,
    ]

    # Assegure que as condições e os valores sejam arrays 1D
    condicoes = [condicionamento.values for condicionamento in condicoes]
    values = [valor.values for valor in values]

    # Cria a nova coluna com np.select
    df['sigma_adm (kPa)'] = np.select(condicoes, values, default=np.nan)

    return df


### Exemplo

In [24]:
import pandas as pd

df = pd.DataFrame({'solo': ['argila', 'areia'], 'spt': [40,20]})
sigma_adm = tensao_adm_solo(df)
print(sigma_adm)

# import pandas as pd
# df = pd.DataFrame({'solo': 'argila', 'spt': 40})

# sigma_adm = tensao_adm_solo(df)
# print(f"Tensão admissível do solo: {sigma_adm}")

     solo  spt  sigma_adm (kPa)
0  argila   40            800.0
1   areia   20            500.0


## Verificação geométrica do balanço

Para garantir o comportamento rígido da sapata e assim permitir a verificação simplificada do método CEB-70, os comprimentos dos balanços da sapata (distância entre a lateral do pilar e a lateral da sapata) devem respeitar o intervalo:

$$
\frac{h}{2} \leq C \leq 2\cdot h 
$$

Onde:
* $C$: representa o comprimento do balanço em cada direção
* $h$: representa a altura da sapata.

A verificação consiste em calcular os balaços nas direções x e y, dados por:

$$ 
Cap = \frac{h_x - a_p}{2}  \quad \text{ e } \quad Cbp = \frac{h_y - b_p}{2}
$$

Onde:
* $h_x$ e $h_y$: dimensões da sapata em x e y, respectivamente
* $a_p$ e $b_p$: dimensões do pilar em x e y, respectivamente

Em seguida, o comprimento do balanço deve ser verificado em relação ao intervalo apresentado acima.

### Função em Python

In [ ]:
def restricao_geometrica_balanco_pilar_sapata(h_x: float, h_y: float, h_z: float, a_p: float, b_p: float) -> tuple[float, float, float , float]:
    """
    Esta função calcula o balanço da sapata e verifica se esta apto a ser calculado de acordo com o método CEB-70

    args:
        h_x (float): dimensões da sapata em x (m)
        h_y (float): dimensões da sapata em y (m)  
        a_p (float): dimensões dos pilares em x (m)
        b_p (float): dimensões dos pilares em y (m)
        
    returns:
        result (float): valor da penalidade (admensional)
    """

    # Balanço na direção X
    cap = (h_x - a_p) / 2

    # Balanço na direção Y
    cbp = (h_y - b_p) / 2
    
    # Restrições laterais do balanço
    g_0 = cap / (2 * h_z) - 1
    g_1 = cbp / (2 * h_z) - 1
    g_2 = (h_z / 2) / cap - 1
    g_3 = (h_z / 2) / cbp - 1

### Exemplo

In [ ]:
#Exemplo de uso da função geométrica do balanço
h_x = 0.9
h_y = 0.6
h_z = 0.4
a_p = 0.3
b_p = 0.19

# Balanço na direção X
cap = (h_x - a_p) / 2

# Balanço na direção Y
cbp = (h_y - b_p) / 2

# Restrições laterais do balanço
g_0 = cap / (2 * h_z) - 1
g_1 = cbp / (2 * h_z) - 1
g_2 = (h_z / 2) / cap - 1
g_3 = (h_z / 2) / cbp - 1

print(f"As dimensões do balanço em x e y, são respectivamente: {cap:.2f} metros e {cbp:.2f} metros.")
print(f"As verificações de comprimento minimo do balanço em x e y, são respectivamente: { g_2:.2f} e {g_3:.2f}.")  
print(f"As verificações de comprimento maximo do balanço em x e y, são respectivamente: {g_0:.2f} e {g_1:.2f}.")

As dimensões do balanço em x e y, são respectivamente: 0.30 metros e 0.20 metros.
As verificações de comprimento minimo do balanço em x e y, são respectivamente: -0.33 e -0.02.
As verificações de comprimento maximo do balanço em x e y, são respectivamente: -0.62 e -0.74.


## Verificação geométrica pilar-sapata

A função' da sapata é transmitir os esforços da superestrutura ao solo. Para tnato, é imprescindível que suas dimensões sejam suficientes para abranger totalmente o pilar. Isso significa que as dimensões da sapata devem ser maiores que as dimensões dos pilares.

$$
a_p < h_x \quad \text{ e } \quad b_p < h_y
$$ 
Onde:
* $h_x$ e $h_y$: dimensões da sapata em x e y, respectivamente.
* $a_p$ e $b_p$: dimensões dos pilares em x e y, respectivamente


### Função em Python

In [ ]:
def restricao_geometrica_pilar_sapata(h_x: float, h_y: float, a_p: float, b_p: float) -> tuple[float, float]:
    """
    Esta função verifica se a dimensão do pilar é maior ou menor que a da sapata

    args:
        h_x (float): dimensões da sapata em x (m)
        h_y (float): dimensões da sapata em y (m)  
        a_p (float): dimensões dos pilares em x (m)
        b_p (float): dimensões dos pilares em y (m)
        
    returns:
        result (float): valor da penalidade (admensional)
    """

        
    #Restrição da dimensão do pilar em relação a dimensão da sapata
    g_4 = a_p / h_x - 1
    g_5 = b_p / h_y - 1

    return g_4, g_5

### Exemplo  

In [14]:
#exemplo da função de verificação das dimensões dos pilares e sapatas
h_x = 0.9
h_y = 0.6
a_p = 0.3
b_p = 0.19

g_4 = a_p / h_x - 1
g_5 = b_p / h_y - 1

print(f"Os valores para verificação em x e y, são respectivamente: {g_4:.2f}, {g_5:.2f}.")

Os valores para verificação em x e y, são respectivamente: -0.67, -0.68.


## Verificação de punção

A verificação da resistência à punção é fundamental para garantir que os esforços transmitidos pelo pilar não provoque ruptura do concreto nas regiões críticas da sapata. Essa verificação segue os crítérios na NBR 6118:2023, considerando a fundação de forma analoga a uma laje. 
<br>Para  isso, necessita-se comparar a tensão solicitante com a tensão resistente nas duas regiões criticas C e C':

* C: borda do pilar
* C': regiao crítica localizada a uma distância 2d da borda do pilar onde d é a altura útil da sapata.

Alem dessa comparação, é realizado as seguintes verificações:

* Se o perímetro C' esta contido dentro da sapata.
* Se a taxa de armadura aderente atende o exigido pela norma. (Falta calcular ela no código)
* Se o coeficiente de escala de punção ke, esta dentro do limite normativo.

Tensão solicitante no perímetro em C em MPa:
$$
\tau_{sd2} = \frac{0,001 \cdot f_z}{2 \cdot a_p + b_p \cdot d}
$$
Onde:

* $f_z$: esforço axial (KN)
* $a_p$ e $b_p$: dimensões dos pilares em x e y, respectivamente (m)
* $d$: altura util do pilar (m)

Tensão resistente no perímetro em C em MPa:
$$
\tau_{rd2} = 0,27 \cdot \left(1 - \frac{f_{ck}}{250}\right) \cdot f_{cd}
$$
Onde:
* $f_{ck}$: 
* $f_{cd}$: 

Tensão solicitante no perímetro C' em MPa: (o fator 0,001 converte f-z para compatibilização de unidades em mpa)
$$
\tau_{sd1} = \frac{0,001 \cdot f_z}{u \cdot d} + \frac{k_x \cdot m_x \cdot 0,001}{w_{px} \cdot d} + \frac{k_y \cdot m_y \cdot 0,001}{w_{py} \cdot d}
$$

onde:

* $f_z$:esforço axial (KN)
* $u$: perímetro crítico 
* $k_x$ e $k_y$: coeficiente de transmissão de momento nas direções x e y.
* $M_x$ e $M_y$: momento de cálculo no plano perpendicular à borda livre na direção de x e y.
* $W_{px} $W_{py}$: É o módulo de resistência plástica perpendicular à borda livre, calculado para o perímetro u na direção de x e y.
* $d$: altura util do pilar (m)

Tensão resistente no perímetro C'em MPa:
$$ 
\tau_{rd1} = 0,13 \cdot k_e \cdot \left(100 \cdot \rho \cdot f_{ck} \right)^{\frac{1}{3}} + 0,1 \cdot \sigma_{cp}
$$
Onde:
* $f_{ck}$: Resistência característica a compressão do concreto (MPa)??
* $ρ$: 
* $k_e$: Coeficiente de escala de punção
* $σ_{cp}$: Tensão devido aos efeitos da protensão do concreto

Variáveis auxiliares:
* perímetro crítico u:
$$
u = 2 \cdot (a_p + b_p + \pi \cdot d)
$$

* O módulo de resistência plástica no perimetro crítico:
    Direção x:
$$
w_{px} = b_p^2 / 2 + b_p \cdot a_p + 4 \cdot a_p \cdot d + 16 \cdot d^2 + 2 \cdot \pi \cdot b_p \cdot d
$$

    Direção y:
$$
...
$$

* O coeficiente de deslocamento relativo k_e:
$$
k_e = 1 + \sqrt{\frac{20}{d \cdot 100}}
$$

Os valores de kx e ky são definidos por interpolação a partir da tabela 19.2 da NBR 6118:2023, conforme a relação b_p / a_p e a_p / b_p.


### Função python

In [ ]:
def restricao_puncao(h_x: float, h_y: float, h_z: float, a_p: float, b_p: float, f_z: float, m_x: float, m_y: float, ro: float, cob: float, fck: float, fcd: float) -> tuple[float, float, float, float, float, float, float,]:
    """
    Esta função  calcula a tensão resistente e solicitante que há na borda do pilar e no perímetro crítico C' da sapata
    em seguida contempla a verificações de restrição de acordo com a NBR6118-2023

    Args:
        h_x (float): Altura da sapata na direção x (m)
        h_y (float): Altura da sapata na direção y (m)
        h_z (float): Altura util da sapata (m)
        a_p (float): Comprimento do pilar (m)
        b_p (float): Largura do pilar (m)
        f_z (float): Esforço axial (KN)
        m_x (float): Momento fletor na direção x (KN.m)
        m_y (float): Momento fletor na direção y (KN.m)
        ro (float): Densidade do concreto (adimensional)
        cob (float): Comprimento de recobramento da sapata (m) 
        fck (float): Resistência característica a compressão do concreto (MPa)
        fcd (float): Resistência de projeto a compressão do concreto (MPa)

    Returns:
       g6, g7, g8, g9, g10, g11, g12 (tuple[float, float, float, float, float, float, float]): verificação das restrições 
       da linha critica, da taxa de aço de flexão aderenente, da tensão de protensão e das tensões em C e C'
    """
    d = h_z - cob #altura util da sapata
    sigma_cp = 0 # tensão a mais devido a efeitos da protensão do concreto <= 3,5 MPa, depois criar uma def para calcular essa tensão!
    ke = 1 + math.sqrt(20 / (d * 100))  
    kx = interpolar_kx(a_p, b_p)
    ky = interpolar_ky(a_p, b_p)
    
    wpx = b_p**2 / 2 + b_p * a_p + 4 * a_p * d + 16 * d**2 + 2 * math.pi * b_p * d #módulo de resistencia plastica no perímetro crítico na direção x
    wpy = a_p**2 / 2 + a_p * b_p + 4 * b_p * d + 16 * d**2 + 2 * math.pi * a_p * d #módulo de resistencia plastica no perímetro crítico na direção y
    u = 2 * (a_p + b_p + math.pi * d) # perímero do contorno C'
    
    talsd2 = 0.001 * f_z / (2 * a_p + b_p * d) # (MPa)
    talrd2 = 0.27 * (1 - fck / 250) * fcd # (MPa)
    talsd1 = (0.001 * f_z) / (u * d) + kx * m_x * 0.001 / (wpx * d) + ky * m_y * 0.001 / (wpy * d) # (MPa)
    talrd1 = 0.13 * ke * (100 * ro * fck) ** (1 / 3) + 0.1 * sigma_cp # (MPa)

    g_6 = (h_x - a_p) / 4 * d - 1 #4 * d / (h_x - a_p) - 1 #se a area crítica esta dentro da sapata em x adiciona punição (preciso verificar isso!)
    g_7 = (h_y - b_p) / 4 * d - 1 #4 * d / (h_y - b_p) - 1 #se a area crítica esta dentro da sapata em y adiciona punição (preciso verificar isso!)
    g_8 = ro / 0.02 - 1 # taxa de aço de flexão aderente, precisa ser calculado
    g_9 =  ke / 2 - 1 
    g_10 = sigma_cp / 3.5 - 1
    g_11 = talsd1 / talrd1 - 1
    g_12 = talsd2 / talrd2 - 1
    
    return  g_6, g_7, g_8, g_9, g_10, g_11, g_12

### Exemplo

In [24]:
# Exemplo da função de verificação de punção
from itertools import combinations
import numpy as np
import pandas as pd
import math

h_x = 0.9
h_y = 0.6
h_z = 0.3
a_p = 0.3
b_p = 0.19

f_z= 10
m_x= 5
m_y= 4

ro = 0.01 #esse valor deve ser calculado
cob = 0.1
fck = 20
fcd = 25

d = h_z - cob #altura util da sapata
sigma_cp = 0 # tensão a mais devido a efeitos da protensão do concreto <= 3,5 MPa, depois criar uma def para calcular essa tensão!
ke = 1 + math.sqrt(20 / (d * 100))  
kx=0.6
ky=0.6
 
wpx = b_p**2 / 2 + b_p * a_p + 4 * a_p * d + 16 * d**2 + 2 * math.pi * b_p * d #módulo de resistencia plastica no perímetro crítico na direção x
wpy = a_p**2 / 2 + a_p * b_p + 4 * b_p * d + 16 * d**2 + 2 * math.pi * a_p * d #módulo de resistencia plastica no perímetro crítico na direção y
u = 2 * (a_p + b_p + math.pi * d) # perímero do contorno C'

talsd2 = 0.001 * f_z / (2 * a_p + b_p * d) # (MPa)
talrd2 = 0.27 * (1 - fck / 250) * fcd # (MPa)
talsd1 = (0.001 * f_z) / (u * d) + kx * m_x * 0.001 / (wpx * d) + ky * m_y * 0.001 / (wpy * d) # (MPa)
talrd1 = 0.13 * ke * (100 * ro * fck) ** (1 / 3) + 0.1 * sigma_cp # (MPa)

g_6 = (h_x - a_p) / 4 * d - 1 #4 * d / (h_x - a_p) - 1 #se a area crítica esta dentro da sapata em x adiciona punição (preciso verificar isso!)
g_7 = (h_y - b_p) / 4 * d - 1 #4 * d / (h_y - b_p) - 1 #se a area crítica esta dentro da sapata em y adiciona punição (preciso verificar isso!)
g_8 = ro / 0.02 - 1 # taxa de aço de flexão aderente, precisa ser calculado
g_9 =  ke / 2 - 1 
g_10 = sigma_cp / 3.5 - 1
g_11 = talsd1 / talrd1 - 1
g_12 = talsd2 / talrd2 - 1

print(f"Tensão solicitante em C e C' respectivamente: {talsd1:.2f}, {talsd2:.2f}")
print(f"Tensão resistente em C e C' respectivamente: {talrd1:.2f}, {talrd2:.2f}")
print(f"Verificação de restrições: g_6={g_6:.2f}, g_7={g_7:.2f}, g_8={g_8:.2f}, g_9={g_9:.2f}, g_10={g_10:.2f}, g_11={g_11:.2f}, g_12={g_12:.2f}")

Tensão solicitante em C e C' respectivamente: 0.04, 0.02
Tensão resistente em C e C' respectivamente: 0.71, 6.21
Verificação de restrições: g_6=-0.97, g_7=-0.98, g_8=-0.50, g_9=0.00, g_10=-1.00, g_11=-0.94, g_12=-1.00


## Tensão atuante

Para definir a tensão atuante em cada sapata, é necessária considerar todas as combinações de esforços provenientes da superestrutura. A partir dessas combinações, identifica-se aquela que resulta na combinação mais desfavorável em temos de tensão.
A tensão de contato entre a sapata e o solo é calculado em função dos esforços verticais e momentos fletores atuant'es, conforme a equação abaixo:
$$
\begin{align*}
\sigma_{max} &= \sigma_{fz} \cdot \left(1 + \text{aux}_{mx} + \text{aux}_{my} \right) \\
\sigma_{\text{min}} &= \sigma_{fz} \cdot \left(1 - \text{aux}_{mx} - \text{aux}_{my} \right)
\end{align*}
$$
Onde:
$$
\sigma_{fz} = \frac{f_z}{h_x \cdot h_y}
$$
é a tensão devido a carga vertical f_z.
<br>
e
$$
\text{aux}_{mx} = \frac{6 \cdot m_x}{f_z \cdot h_x}
$$

$$
\text{aux}_{my} = \frac{6 \cdot m_y}{f_z \cdot h_y}
$$
é a tensão máxima e mínima devido a caarga excntrica, respectivamente. 

### Função Python

In [25]:
# FUnção que cálcula a tensão solicitante
def calcular_sigma_max(f_z: float, m_x: float, m_y: float, h_x: float, h_y: float) -> tuple[float, float]:
    """
    Esta função determina a tensão máxima e a tensão mínima solicitante na fundação rasa do tipo sapata

    Args
    f_z: Carregamento na direção z, da combinação mais desfavorável (kN)
    m_x: Momento em x da combinação mais desfavorável (kN.m)
    m_y: Momento em y da combinação mais desfavorável (kN.m)

    Returns
    sigma_max: Tensão máxima que age na sapata (kPa)
    sigma_min: Tensão minima que age na sapata (kPa)

    """
    
    m_x = abs(m_x)
    m_y = abs(m_y)
    sigma_fz = f_z / (h_x * h_y)
    aux_mx = 6 * (m_x / f_z) / h_x
    aux_my = 6 * (m_y / f_z) / h_y
    
    return (sigma_fz) * (1 + aux_mx + aux_my), (sigma_fz) * (1 - aux_mx - aux_my)

### Exemplo de uso

In [ ]:
#exemplo para calculo de tensão máx e min
m_x = 10
m_y = 5
f_z = 30
h_x = 2
h_y = 1

sigma_max,sigma_min = calcular_sigma_max(f_z, m_x, m_y, h_x, h_y)
print(f"Tensão Máxima: {sigma_max:.2f} kPa")
print(f"Tensão Minima: {sigma_min:.2f} kPa")

Tensão Máxima: 45.00 
Tensão Minima: -15.00 


## Combinação de esforços

### Função Python

In [ ]:
#errado

df1 = pd.DataFrame({
        'Fz-c1': [485.90, 885.80, 1314.00],
        'Mx-c1': [-0.30, -1.00, -3.90],
        'My-c1': [4.10, 9.20, 20.40],
        'Fz-c2': [511.60, 912.10, 1696.40],
        'Mx-c2': [-32.40, -65.40, -214.10],
        'My-c2': [-0.40, 0.10, 0.80],
        'Fz-c3': [511.60, 915.90, 1698.90],
        'Mx-c3': [-32.40, -40.00, -214.60],
        'My-c3': [-0.40, 0.10, 0.70],
    })

def obj_ic_fundacoes(h_x, h_y,n_comb, df1):
    h_x = 0.6
    h_y = 0.6
    n_comb = 3
    
    
    t_max = []
    t_min = []
    fz_list = []
    mx_list = []
    my_list = []
    t_value = []
    print(df1)
    
    # determinando a combinação mais desfavorável
    for idx, row in df1.iterrows():
        for i in range(1, n_comb+1):
            aux = f'c{i}'
            t_max_aux, t_min_aux = calcular_sigma_max(row[f'Fz-{aux}'], row[f'Mx-{aux}'], row[f'My-{aux}'], h_x, h_y)
            t_max.append(t_max_aux)
            t_min.append(t_min_aux)
            # if para garantir que os valores de FZ mx e my sejam correspondente à combinação mais desfavorável
            if t_max_aux >= t_max[-1]:
                fz_aux = row[f'Fz-{aux}']
                mx_aux = row[f'Mx-{aux}']
                my_aux = row[f'My-{aux}']
                fz_list.append(fz_aux)
                mx_list.append(mx_aux)
                my_list.append(my_aux)
        f_z = max(fz_list)
        m_x = max(mx_list)
        m_y = max(my_list)
        t_max_value = max(t_max)
        t_min_value = min(t_min)
        t_value = max(abs(t_max_value), abs(t_min_value))
 
    return f_z, m_x, m_y, t_value

f_z, m_x, m_y, t_value = obj_ic_fundacoes([1, 2, 0.5], [0.6, 1, 0.9], 3, df1)

print("f_z:", f_z)
print("m_x:", m_x)
print("m_y:", m_y)
print("t_value:", t_value)



    Fz-c1  Mx-c1  My-c1   Fz-c2  Mx-c2  My-c2   Fz-c3  Mx-c3  My-c3
0   485.9   -0.3    4.1   511.6  -32.4   -0.4   511.6  -32.4   -0.4
1   885.8   -1.0    9.2   912.1  -65.4    0.1   915.9  -40.0    0.1
2  1314.0   -3.9   20.4  1696.4 -214.1    0.8  1698.9 -214.6    0.7
f_z: 1698.9
m_x: -0.3
m_y: 20.4
t_value: 10699.722222222224


## Verificação de sobreposição

Esta aplicação permite o dimensionament completo de projetos de fundação com multiplos elementos Nesse contexto é essencial garantir que não haja sobreposição das sapatas, pois a ocorrência de sobreposição impede o funcionamento das sapatas isoladas, exigindo outro tipo de solução como sapata associadas.
<br> Para verificar essa condição e permitir sua incorporação em um algorítmo de otimização, considera-se a área de projeção de cada elememnto de fundação no plano xy. A sobreposição é avalida por meio da analises das coordenadas das vertices da fundação, alcançado por meio do conjunto de equações a seguir:

* Coordenadas dos vértices da sapata:
$$
x_{i,{min}} = x_i - \frac{h_x}{2}  , \quad x_{i,\text{max}} = x_i + \frac{h_x}{2} \\
$$
$$
y_{i,\text{min}} = y_i - \frac{h_y}{2} , \quad y_{i,\text{max}} = y_i + \frac{h_y}{2}
$$

* Comprimento da região sobreposta nas direções x e y:
$$ 
\text{overlap}_x = \max\left(0, \min(x_{i,\text{max}}, x_{j,\text{max}}) - \max(x_{i,\text{min}}, x_{j,\text{min}})\right)
$$
$$
\text{overlap}_y = \max\left(0, \min(y_{i,\text{max}}, y_{j,\text{max}}) - \max(y_{i,\text{min}}, y_{j,\text{min}})\right)
$$

* Área de sobreposição:
$$
\text{area}_{\text{overlap}} = \text{overlap}_x \cdot \text{overlap}_y
$$

O código implementado percorre cada elemento i e o compara com todos os demais elementos j. Caso haja sobreposição, a área de sobreposição é computada e somada ao total. O valor final corresponde a soma das áreas de sobreposição.

### Função em Python

In [ ]:
def restricao_geometrica_sobreposicao(df, h_x, h_y, idx):
    """
    Verifica a soma da área de sobreposição da sapata atual (em sapata_index)
    com todas as outras sapatas do DataFrame.

    Retorna:
        area_total_sobreposta (float): penalização proporcional à área de sobreposição.
    """
    area_total = 0
    xi, yi = df.loc[idx, 'xg (m)'], df.loc[idx, 'yg (m)']

    xi_min, xi_max = xi - h_x / 2, xi + h_x / 2
    yi_min, yi_max = yi - h_y / 2, yi + h_y / 2

    for j, row in df.iterrows():
        if j == idx:
            continue  # Ignorar a própria sapata

        xj, yj = row['xg (m)'], row['yg (m)']
        xj_min, xj_max = xj - h_x / 2, xj + h_x / 2
        yj_min, yj_max = yj - h_y / 2, yj + h_y / 2

        # Calcular sobreposição
        overlap_x = max(0, min(xi_max, xj_max) - max(xi_min, xj_min))
        overlap_y = max(0, min(yi_max, yj_max) - max(yi_min, yj_min))
        area_overlap = overlap_x * overlap_y

        area_total += area_overlap

    return area_total / (h_x * h_y)  # penalização normalizada

### Exemplo

In [30]:
#exemplo de uso da função de sobreposição
xg1= 3
yg1= 4
xg2= 4
yg2= 5
h_x1= 2
h_y1= 2
h_x2= 2 
h_y2= 2

xi_min, xi_max = xg1 - h_x1 / 2, xg1 + h_x1 / 2
yi_min, yi_max = yg1 - h_y1 / 2, yg1 + h_y1 / 2

xj_min, xj_max = xg2 - h_x2 / 2, xg2 + h_x2 / 2
yj_min, yj_max = yg2 - h_y2 / 2, yg2 + h_y2 / 2

overlap_x = max(0, min(xi_max, xj_max) - max(xi_min, xj_min))
overlap_y = max(0, min(yi_max, yj_max) - max(yi_min, yj_min))
area_overlap = overlap_x * overlap_y

print(f"Comprimento de sobreposição em x e y, respectivament: {overlap_x:.2f}, {overlap_y:.2f}")
print(f"A área de sobreposição é: {area_overlap:.2f}")


Comprimento de sobreposição em x e y, respectivament: 1.00, 1.00
A área de sobreposição é: 1.00
